In [21]:
import torch
from PIL import Image
import torchvision.transforms as transforms
import numpy as np
import json
import requests
import matplotlib.pyplot as plt
import warnings

warnings.filterwarnings('ignore')
%matplotlib inline


if torch.cuda.is_available():
    device = torch.device("cuda") 
    !nvidia-smi
else:
    device = torch.device("cpu")

print(f'Using {device}')

Using cpu


In [1]:
! pip install timm

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
from torchvision.models.feature_extraction import create_feature_extractor

model_type = "GPUNet-0" # select one from above
precision = "fp32"
gpunet = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_gpunet', pretrained=True, model_type=model_type, model_math=precision)
# gpunet = 


Using cache found in C:\Users\Maryk/.cache\torch\hub\NVIDIA_DeepLearningExamples_torchhub


model_type= GPUNet-0
mapped model latency= 0.65ms
testing:  batch= 1  latency= 0.65ms  gpu= GV100


0B [00:00, ?B/s]

In [11]:
PATH = '../Models/gpunet.pth'

torch.save(gpunet,PATH)

In [18]:
from torch import load
gg = load('D:/Project/YOLO v6/YOLOv6/Models/gpunet.pth')

In [20]:
gg

GPUNet(
  (network): Sequential(
    (head: 2): prologue_i3_o32_s2_swish
    (stage: 1 layer3): conv_k3_i32_o32_s1_relu
    (stage: 1 layer4): conv_k3_i32_o32_s1_relu
    (stage: 2 layer5): er_k3_e5_i32_o32_s2_relu_se_False
    (stage: 2 layer6): er_k3_e5_i32_o32_s1_relu_se_False
    (stage: 3 layer7): er_k3_e5_i32_o64_s2_relu_se_False
    (stage: 3 layer8): er_k3_e5_i64_o64_s1_relu_se_False
    (stage: 3 layer9): er_k3_e5_i64_o64_s1_relu_se_False
    (stage: 4 layer10): irb_k3_e5_i64_o256_s2_swish_se_False
    (stage: 4 layer11): irb_k3_e5_i256_o256_s1_swish_se_False
    (stage: 4 layer12): irb_k3_e5_i256_o256_s1_swish_se_False
    (stage: 6 layer13): irb_k3_e5_i256_o704_s2_relu_se_True
    (stage: 6 layer14): irb_k3_e5_i704_o704_s1_relu_se_True
    ( layer15): epilogue_i704_o1280_s1_relu
  )
)

In [26]:
# for name, layer in gpunet.named_modules():
#     print(name)

import torch.nn as nn

gpunet.to(device)
gpunet.eval()

for name, layer in gpunet.named_modules():
    if isinstance(layer, nn.ReLU):
        print(name, layer)

network.stage: 1 layer3.act1 ReLU(inplace=True)
network.stage: 1 layer4.act1 ReLU(inplace=True)
network.stage: 2 layer5.act1 ReLU(inplace=True)
network.stage: 2 layer6.act1 ReLU(inplace=True)
network.stage: 3 layer7.act1 ReLU(inplace=True)
network.stage: 3 layer8.act1 ReLU(inplace=True)
network.stage: 3 layer9.act1 ReLU(inplace=True)
network.stage: 6 layer13.act1 ReLU(inplace=True)
network.stage: 6 layer13.act2 ReLU(inplace=True)
network.stage: 6 layer13.se.act1 ReLU(inplace=True)
network.stage: 6 layer14.act1 ReLU(inplace=True)
network.stage: 6 layer14.act2 ReLU(inplace=True)
network.stage: 6 layer14.se.act1 ReLU(inplace=True)
network. layer15.net.2 ReLU(inplace=True)


In [67]:
return_nodes = {
    "layer15.net.2": "layer14"
}
model2 = create_feature_extractor(gpunet, eval_return_nodes={"layer4":"0"} , train_return_nodes={})
# intermediate_outputs = model2(x)
# gpunet.modules

TypeError: rand() received an invalid combination of arguments - got (Proxy, device=Attribute, dtype=Attribute), but expected one of:
 * (tuple of ints size, *, torch.Generator generator, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, torch.Generator generator, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, Tensor out, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)
 * (tuple of ints size, *, tuple of names names, torch.dtype dtype, torch.layout layout, torch.device device, bool pin_memory, bool requires_grad)


In [74]:
from torchvision.models._utils import IntermediateLayerGetter
model2 = create_feature_extractor(gpunet, return_nodes={"layer4":"0"})

AttributeError: 'generator' object has no attribute 'training'

In [33]:
import torchvision
model = torchvision.models.resnet50(pretrained=True)
model1 = create_feature_extractor(model, return_nodes={'layer4': '0'})
model2 = IntermediateLayerGetter(model, return_layers={'layer4': '0'})

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to C:\Users\Maryk/.cache\torch\hub\checkpoints\resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [07:30<00:00, 228kB/s]   


In [19]:
torch.__version__

'2.1.2+cu118'

In [122]:
# mds = list(gpunet.modules())
mdl = mds[2]
print(mdl)
create_feature_extractor((mdl), return_nodes=[]) 

prologue_i3_o32_s2_swish


Prologue()

In [108]:
c = 0
for i in mds:
    print(c)
    c+= 1
    print(i)

0
GPUNet(
  (network): Sequential(
    (head: 2): prologue_i3_o32_s2_swish
    (stage: 1 layer3): conv_k3_i32_o32_s1_relu
    (stage: 1 layer4): conv_k3_i32_o32_s1_relu
    (stage: 2 layer5): er_k3_e5_i32_o32_s2_relu_se_False
    (stage: 2 layer6): er_k3_e5_i32_o32_s1_relu_se_False
    (stage: 3 layer7): er_k3_e5_i32_o64_s2_relu_se_False
    (stage: 3 layer8): er_k3_e5_i64_o64_s1_relu_se_False
    (stage: 3 layer9): er_k3_e5_i64_o64_s1_relu_se_False
    (stage: 4 layer10): irb_k3_e5_i64_o256_s2_swish_se_False
    (stage: 4 layer11): irb_k3_e5_i256_o256_s1_swish_se_False
    (stage: 4 layer12): irb_k3_e5_i256_o256_s1_swish_se_False
    (stage: 6 layer13): irb_k3_e5_i256_o704_s2_relu_se_True
    (stage: 6 layer14): irb_k3_e5_i704_o704_s1_relu_se_True
    ( layer15): epilogue_i704_o1280_s1_relu
  )
)
1
Sequential(
  (head: 2): prologue_i3_o32_s2_swish
  (stage: 1 layer3): conv_k3_i32_o32_s1_relu
  (stage: 1 layer4): conv_k3_i32_o32_s1_relu
  (stage: 2 layer5): er_k3_e5_i32_o32_s2_relu_se_

In [116]:
c = 0
for i in model.modules():
    print(c)
    c+= 1
    print(i)

0
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1

use hook forward

In [123]:
def get_features(name):
    def hook(model, input, output):
        features[name] = output.detach()

    return hook

In [130]:
c3 = mds[79]
c4 = mds[93]
c5 = mds[116]

c3.register_forward_hook(get_features("c3"))
c4.register_forward_hook(get_features("c4"))
c5.register_forward_hook(get_features("c5"))

In [132]:
! pip install validators

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [135]:
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_convnets_processing_utils')
uris = [
    'http://images.cocodataset.org/test-stuff2017/000000024309.jpg',
    'http://images.cocodataset.org/test-stuff2017/000000028117.jpg',
    'http://images.cocodataset.org/test-stuff2017/000000006149.jpg',
    'http://images.cocodataset.org/test-stuff2017/000000004954.jpg',
]

batch = torch.cat(
    [utils.prepare_input_from_uri(uri) for uri in uris]
).to(device)

if precision == "fp16":
    batch = batch.half()
    
print("Ready to run inference...")


In [144]:
##### FEATURE EXTRACTION LOOP

# placeholders


# placeholder for batch features
features = {}

with torch.no_grad():
    output = torch.nn.functional.softmax(gpunet(batch), dim=1)

    # add feats and preds to lists
FEATS.append([features["c3"].cpu().numpy() , features["c4"].cpu().numpy() , features["c5"].cpu().numpy()])


In [145]:
FEATS = np.concatenate(FEATS)
print(FEATS.shape)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (3, 4) + inhomogeneous part.

In [146]:
FEATS

[[array([[ 0.2259801 , -0.15294072, -1.512263  , ..., -1.2224617 ,
          -0.90127724,  0.02961297],
         [ 0.09717762,  0.67677516,  0.5848702 , ...,  0.15049517,
           1.4610047 , -0.17677425],
         [ 0.6227139 ,  0.1135464 ,  0.521129  , ..., -1.1161091 ,
          -0.3224512 , -1.0071607 ],
         [ 0.07814017,  0.33386844,  1.6657013 , ...,  0.2418642 ,
           1.3473537 ,  1.6984595 ]], dtype=float32),
  array([[[[-2.16793942e+00, -1.92733169e+00,  6.87612653e-01, ...,
             1.31629989e-01, -1.93996787e+00, -1.28968441e+00],
           [-7.63811231e-01, -1.37549326e-01, -8.67556036e-01, ...,
            -1.15073383e+00, -9.61326540e-01, -2.33602393e-02],
           [ 7.05757663e-02, -5.99115491e-01, -1.92986655e+00, ...,
            -1.91740251e+00, -1.47812116e+00, -1.20598562e-01],
           ...,
           [ 9.21590030e-01,  2.42263332e-01, -5.06356478e-01, ...,
             4.87339687e+00,  7.50002205e-01, -1.07349372e-02],
           [ 2.16605115

In [14]:

model_type = "GPUNet-0" # select one from above
precision = "fp32"
path = 'C:/Users/Maryk/.cache/torch/hub'

gpunet,a = torch.hub.load(path , model ='nvidia_gpunet', pretrained=True, model_type=model_type, model_math=precision)


ValueError: not enough values to unpack (expected 2, got 1)